# Complex REST APIs

This lab is designed to aid understanding the wonderful world of REST, and how one can interact with a Create/Read/Update/Delete stack (CRUD).

## What is a REST API?
A *Representational state transfer* (REST) API is a way of structuring an API so that, through the use of HTTP verbs (like GET and POST) one can get data. There are many HTTP verbs, but you are likely to run into the following the most
+ *GET* - used when you are attempting to access a resource
+ *POST* - used when creating a new instance of a resource
+ *PUT* - used when attempting a _complete update_ of a resource
+ *PATCH* - used when attempting a _partial update_ of a resource
+ *DELETE* - used when deleting a resource

_(For a complete list of HTTP verbs for those who are interested, the [Mozilla documentation](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) is very well written)_

The URLs of the REST service are built to fit hand-in-hand with these verbs to create a logical sense of direction and destination. For example, consider the following sentences:
> "I would like to get a list of all messages in the database."  
> "I would like to create a new message"  
> "I have a message ID, and I would like to get that message's information"

We can turn all three of those into REST endpoints very easily:

+ GET `/messages`
+ POST `/message`
+ GET `/message/:messageId`  
_Note: This lab will be using the convention of prefixing "slug" values (values to be replaced by a program) with a colon. In the final example above, understand that `:messageId` is not literally part of the URL, but rather is to be replaced with a message ID._

In this lab, we will be interacting with an example-built message board system using Python's built-in HTTP libraries.

> A quick note before we go forward - in the real world, you will normally see routes like we've shown in the example above, where the route is named after the resource you are getting/updating. We've chosen a solution for hosting HTTP routes for these labs that unfortunately doesn't allow us to name the routes as one normally would. Please understand that, where we have `/ca-get-messages` and `/ca-post-message`, the real-world ideal would be named just `/messages` and `/message` respectively.

## `GET`-ting information

Our example message-board system has the following route for getting messages:  
+ GET `/ca-get-messages` - gets all messages in our system. Returns a 200 with all of the messages in the system, or 500 if an error was encountered.

When accessing this route through Postman (a good tool for testing HTTP endpoints), we get the following response:  
![Example /messages output](./assets/lab7/get-postman.png)  

Let's get this information in Python! We will be using the `requests` module for HTTP request.  
The following example will get an example JSON response from [httpbin.org](http://httpbin.org/get). You can use the following code as a base for your code to get message from our system.

In [20]:
import requests

req_url = "http://httpbin.org/get"

response = requests.get(req_url)

if response.status_code == 200:
    print(response.text)
else:
    print("Error on request")
    print(response.status_code)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.24.0", 
    "X-Amzn-Trace-Id": "Root=1-5f038f75-af10930880148a040da003dd"
  }, 
  "origin": "34.82.146.253", 
  "url": "http://httpbin.org/get"
}



#### Lab: Getting Started with GET

Use the following code block for your code, to perform a GET request from our endpoint. The URL of the request is already defined for you in the `req_url` variable.

In [16]:
import requests

req_url = "http://us-central1-calabs-dev.cloudfunctions.net:3000/ca-get-messages"

# Your code here!

[{"id":"0c4733c1-6e7b-4a97-b186-2affcb1fb224","message":"Test message from in Jupyter erics test","last_edited":"2020-07-06T20:27:15.000Z"},{"id":"68c77d49-b726-4d86-b455-a19ca095d190","message":"This is a message!","last_edited":"2020-07-04T11:04:46.000Z"},{"id":"81dec8f0-f628-4c37-bed5-aad44061635b","message":"test message","last_edited":"2020-07-04T11:48:37.000Z"},{"id":"bee576ef-7b6d-4d8a-9d4f-88fa251f2365","message":"Test message from in Jupyter erics test","last_edited":"2020-07-06T20:27:15.000Z"}]


# `POST`-ing a new message

When we make a POST request, we usually will need to supply some data as well. Using the `requests` module, this is easy:

```py
import requests

req_url = "https://some-rest-service/some-endpoint"

response = requests.post(req_url, data = {
   "dataField1": "some-data",
   "dataFiled2": "some-other-data"
})
```

As you see above, all we need to do is just supply a `data` keyword-argument after our request URL. Knowing this, let's see what's required to add a new message to our message board.

+ POST `/ca-post-message` - Creates a new message, the content of which is captured in the `message` field. Returns a 201 with ID of the new message created, and 500 for errors.

According to the specification, all we need to provide in our data object is a string field called `message`. Since this is a "create" action, the API will return an HTTP status of `201`. You may recall that our GET request returned a `200` - HTTP responses come along with a status code to better specify what happened on the server-side (this is where a "404" page comes from!). [HTTPStatuses](http://httpstatuses.com) is a fantastic resource for looking up the meaning of HTTP status codes.  

#### Lab: Practicing POST Requests

You now know everything you need to post new messages to the board. Try it out!

> Hint - make sure that you get the messageId from the response, since we'll be using it later when we go to edit it. The code below does so by accessing the `text` field of the response object.

In [17]:
import requests

# Your code here!

req_url = "http://us-central1-calabs-dev.cloudfunctions.net:3000/ca-post-message"

response = requests.post(req_url, data = {
    "message": "Test message from in Jupyter"
})

if response.status_code == 201:
    print(response.text)
else:
    print("There was an error with your request")
    print(response.status_code)

{"messageId":"6b5fe17d-84fe-4015-a82f-7550b47deeaa"}


## `PATCH`-ing previous messages

In the REST world, there are two different HTTP verbs that represent updating data - `PUT` and `PATCH`. They both basically accomplish the same thing - their differences are more semantic than substantitive. `PUT` requests are for requests who require a representation of an entire object to be updated. This means the resource on the server will be completely and wholly replaced by what is being sent in the request. `PATCH` requests, on the other hand, allow for "partial" updates. This means only the fields provided in the request will be modified. There's a [really good Stack Overflow](https://stackoverflow.com/questions/28459418/use-of-put-vs-patch-methods-in-rest-api-real-life-scenarios) post about this if you really want to get into the details.

You may recall from your `GET` experiments that our messages have three fields: `id`, `message`, and `last_updated`. The server automatically generates the `id` field and, since it is the unique identifier for each message, we do not want to change this. The server is also responsible for updating the `last_updated` field, so the only field we need to be sending in our requests is the `message`. As such, we will be using `PATCH` to edit a message.

With the `requests` library, the structure for making a `PATCH` request is identical to `POST`, except you'll want to invoke the `patch` function instead of `post`. Let's see what our API expects in terms of requests:
+ PATCH `/ca-patch-message?messageId=<your-message-id>` - updates the message with the given messageId with the given `message` field. This route will return a 200 if the data was successfully patched, 404 if the given message ID wasn't found, and 500 on other errors.
> Editor's note - as mentioned previously, this syntax is unfortunately not standard. Most services will employ a PATCH `/message/:messageId` instead of what we have here. This discrepancy is due to a limitation with our HTTP hosting service.

#### Lab: Practicing with Patch

Use the following code block to update your previous message to something else. Then, run the `GET` request from earlier to see your updated message!

In [19]:
import requests

message_id = "6b5fe17d-84fe-4015-a82f-7550b47deeaa"

req_url = "http://us-central1-calabs-dev.cloudfunctions.net:3000/ca-patch-message?messageId=" + message_id

response = requests.patch(req_url, data = {
    "message": "Updated test message from in Jupyter!"
})

if response.status_code == 200: 
    print("Message with ID " + message_id + " updated!")
elif response.status_code == 404:
    print("Message with ID " + message_id + " not found")
else:
    print("Couldn't update your message")
    print(response.status_code)
    

Message with ID 6b5fe17d-84fe-4015-a82f-7550b47deeaa updated!


## `DELETE`-ing messages

The final HTTP verb we'll discuss in this lab is the `DELETE` verb. It's pretty obvious what it does - it removes a resource from the system. Let's examine how our message board system handles `DELETE` requests:

+ DELETE `/ca-delete-message?messageId=<your-message-id>` - deletes the message with the corresponding message ID. Returns 200 if the message was deleted, 404 if the message ID wasn't found, and 500 for other errors.

#### Lab: Deleting Data via REST

This request doesn't require any data, so your function should look more akin to your `GET` code. Predictably, you can invoke the `delete` function from `requests` to make an HTTP `DELETE` request.

In [10]:
import requests

message_id = "6a71103c-4249-4703-97de-a43157211272"

req_url = "http://us-central1-calabs-dev.cloudfunctions.net:3000/ca-delete-message?messageId=" + message_id

response = requests.delete(req_url)

if response.status_code == 200: 
    print("Message with ID " + message_id + " deleted!")
elif response.status_code == 404:
    print("Message with ID " + message_id + " not found")
else:
    print("Couldn't update your message")
    print(response.status_code)

Message with ID 6a71103c-4249-4703-97de-a43157211272 deleted!
